In [1]:
import functools

import numpy as np
import tensorflow as tf

In [2]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

24576/13049 [========================================================] - 0s 0us/step


In [3]:
np.set_printoptions(precision=3, suppress = True)

In [6]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [7]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(file_path, batch_size = 5,
                                                  label_name = LABEL_COLUMN, na_value = '?',
                                                  num_epochs = 1, ignore_errors = True, **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [10]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key, value.numpy()))

In [11]:
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'male' b'male' b'female']
age                 : [16. 26.  2. 25. 28.]
n_siblings_spouses  : [1 2 4 1 1]
parch               : [3 0 1 0 0]
fare                : [34.375  8.663 39.688 17.8   15.5  ]
class               : [b'Third' b'Third' b'Third' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton' b'Queenstown']
alone               : [b'n' b'n' b'n' b'n' b'n']


In [12]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouse', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']
temp_dataset = get_dataset(train_file_path, column_names = CSV_COLUMNS)
show_batch(temp_dataset)

sex                 : [b'male' b'male' b'female' b'male' b'female']
age                 : [28.  20.5 45.  36.  40. ]
n_siblings_spouse   : [0 0 1 1 1]
parch               : [0 0 1 2 1]
fare                : [  7.896   7.25  164.867  27.75   39.   ]
class               : [b'Third' b'Third' b'First' b'Second' b'Second']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'y' b'y' b'n' b'n' b'n']


In [14]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']
temp_dataset = get_dataset(train_file_path, select_columns = SELECT_COLUMNS)
show_batch(temp_dataset)

age                 : [32. 28. 48. 21. 35.]
n_siblings_spouses  : [0 8 1 0 0]
class               : [b'Third' b'Third' b'First' b'First' b'First']
deck                : [b'E' b'unknown' b'A' b'D' b'unknown']
alone               : [b'y' b'n' b'n' b'n' b'y']


In [15]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, select_columns = SELECT_COLUMNS, column_defaults = DEFAULTS)
show_batch(temp_dataset)

age                 : [32. 40. 21. 34. 28.]
n_siblings_spouses  : [0. 1. 0. 0. 1.]
parch               : [0. 1. 0. 1. 0.]
fare                : [56.496 39.    10.5   23.    15.85 ]


In [16]:
example_batch, labels_batch = next(iter(temp_dataset))

In [20]:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

In [22]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

[[23.     0.     0.    15.046]
 [18.     0.     0.     9.842]
 [28.     0.     0.     7.75 ]
 [71.     0.     0.    49.504]
 [24.     1.     0.    26.   ]]

[0 1 1 0 1]


In [23]:
show_batch(raw_train_data)

sex                 : [b'male' b'female' b'male' b'female' b'female']
age                 : [40.5 15.  28.  28.  24. ]
n_siblings_spouses  : [0 0 0 2 0]
parch               : [2 0 0 0 2]
fare                : [14.5    7.225  8.05  23.25  16.7  ]
class               : [b'Third' b'Third' b'Third' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'G']
embark_town         : [b'Southampton' b'Cherbourg' b'Southampton' b'Queenstown' b'Southampton']
alone               : [b'n' b'y' b'y' b'n' b'n']


In [49]:
example_batch, labels_batch = next(iter(packed_train_data))

In [41]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [42]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [43]:
show_batch(packed_train_data)

sex                 : [b'female' b'female' b'male' b'male' b'female']
class               : [b'Third' b'Second' b'Third' b'Third' b'Second']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Queenstown' b'Southampton']
alone               : [b'y' b'y' b'y' b'n' b'n']
numeric             : [[28.     0.     0.     7.55 ]
 [34.     0.     0.    13.   ]
 [24.     0.     0.     7.496]
 [ 2.     4.     1.    29.125]
 [ 2.     1.     1.    26.   ]]


데이터 정규화

In [44]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [45]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [46]:
def normalize_numeric_data(data, mean, std):
  return (data-mean)/std

In [47]:
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7eff5d4310e0>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [51]:
example_batch['numeric']

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[26.   ,  0.   ,  0.   ,  7.887],
       [18.   ,  0.   ,  1.   , 23.   ],
       [ 9.   ,  4.   ,  2.   , 31.388],
       [22.   ,  0.   ,  0.   ,  9.   ],
       [28.   ,  0.   ,  0.   , 26.   ]], dtype=float32)>

In [50]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-0.29 , -0.474, -0.479, -0.485],
       [-0.93 , -0.474,  0.782, -0.209],
       [-1.649,  3.001,  2.043, -0.055],
       [-0.61 , -0.474, -0.479, -0.465],
       [-0.13 , -0.474, -0.479, -0.154]], dtype=float32)

In [54]:
CATEGORIES = {
    'sex':['male', 'female'],
    'class':['First', 'Second', 'Third'],
    'deck':['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town':['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone':['y','n']
}

In [55]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(key = feature, vocabulary_list = vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [56]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [59]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


In [62]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns + numeric_columns)

In [63]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 1.     0.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.    -0.29  -0.474
 -0.479 -0.485  1.     0.   ]


In [64]:
 model = tf.keras.Sequential([
                              preprocessing_layer,
                              tf.keras.layers.Dense(128, activation = 'relu'),
                              tf.keras.layers.Dense(128, activation = 'relu'),
                              tf.keras.layers.Dense(1),
 ])

 model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits = True), optimizer = 'adam', metrics = ['accuracy'])

In [65]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [66]:
model.fit(train_data, epochs = 20)

Epoch 1/20
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
126/126 [==============================] - 2s 3ms/step - loss: 0.5093 - accuracy: 0.7225
Epoch 2/20
126/126 [==============================] - 0s 3ms/step - loss: 0.4182 - accuracy: 0.8102
Epoch 3/20
126/126 [==============================] - 0s 2ms/step - loss: 0.4056 - accuracy: 0.8198
Epoch 4/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3898 - accuracy: 0.8373
Epoch 5/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3792 - accuracy: 0.8293
Epoch 6/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3750 - accuracy: 0.8437
Epoch 7/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3644 - accuracy: 0.8373
Epoch 8/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3558 - accuracy: 0.8437
Epoch 9/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3449 - accurac

In [67]:
test_loss, test_accuracy = model.evaluate(test_data)
print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

Consider rewriting this model with the Functional API.
53/53 [==============================] - 0s 2ms/step - loss: 0.4589 - accuracy: 0.8409


Test Loss 0.4589184522628784, Test Accuracy 0.8409090638160706


In [69]:
predictions = model.predict(test_data)

for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  prediction = tf.sigmoid(prediction).numpy()
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))

Predicted survival: 61.24%  | Actual outcome:  DIED
Predicted survival: 16.91%  | Actual outcome:  SURVIVED
Predicted survival: 94.44%  | Actual outcome:  DIED
Predicted survival: 55.36%  | Actual outcome:  DIED
Predicted survival: 47.49%  | Actual outcome:  DIED
